# AlexNet-Prototyp: Training

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

## AlexNet (Model.ipynb)

In [ ]:
# Input
inputs = tf.keras.Input(shape=(224, 224, 3), name="alexnet_input")

# Schicht 1: Convolution
l1 = tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, padding="same")(inputs)
l1 = tf.keras.layers.BatchNormalization()(l1)
l1 = tf.keras.layers.ReLU()(l1)
l1 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)(l1)

# Schicht 2: Convolution
l2 = tf.keras.layers.Conv2D(filters=256, kernel_size=5, strides=1, padding="same")(l1)
l2 = tf.keras.layers.BatchNormalization()(l2)
l2 = tf.keras.layers.ReLU()(l2)
l2 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)(l2)

# Schicht 3: Convolution
l3 = tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1, padding="same")(l2)
l3 = tf.keras.layers.ReLU()(l3)

# Schicht 4: Convolution
l4 = tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1, padding="same")(l3)
l4 = tf.keras.layers.ReLU()(l4)

# Schicht 5: Convolution
l5 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, padding="same")(l4)
l5 = tf.keras.layers.ReLU()(l5)
l5 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)(l5)

# Schicht 6: Dense
l6_pre = tf.keras.layers.Flatten()(l5)

l6 = tf.keras.layers.Dense(units=4096)(l6_pre)
l6 = tf.keras.layers.ReLU()(l6)
l6 = tf.keras.layers.Dropout(rate=0.5)(l6)

# Schicht 7: Dense
l7 = tf.keras.layers.Dense(units=4096)(l6)
l7 = tf.keras.layers.ReLU()(l7)
l7 = tf.keras.layers.Dropout(rate=0.5)(l7)

# Schicht 8: Dense
l8 = tf.keras.layers.Dense(units=1000)(l7)
l8 = tf.keras.layers.Softmax(dtype=tf.float32, name="alexnet_output")(l8)

alexnet = tf.keras.models.Model(inputs=inputs, outputs=l8)

## Callbacks
- **Early Stopping**: Das Training wird beendet, wenn sich eine der überwachten Metriken nicht mehr verbessert, hier *Validation Accuracy*. Sie berechnet, wie oft Vorhersagen mit One-Hot Labels übereinstimmen.
- **Reduce Learning Rate**: Gemäß der Autorspezifikation wird die Lernrate von AlexNet um den faktor 10 reduziert, wenn sich die Genauigkeit der *Loss Validation Accuracy* nicht verbessert. Die *Loss Validation Accuracy* ist die Summe der Fehler, die für jedes Beispiel in Trainings- oder Validierungssätzen gemacht wurden. *Loss* gibt an, wie gut sich ein Modell nach jeder Iteration der Optimierung verhält. *Accuracy* wird verwendet, um die Leistung des Algorithmus zu messen.
- **Tensorboard**: damit werden die Trainings- und Validierungsmetriken auf Tensorboard für die Trainingsanalyse veröffentlicht

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_categorical_accuracy",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_categorical_accuracy",
    factor=0.1,
    patience=2,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=10e-8,
)

tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir="log",
    histogram_freq=2,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
    profile_batch=2,
    embeddings_freq=2,
    embeddings_metadata=None,
)
callbacks = [early_stop, reduce_learning_rate, tensorboard]

## Metriken: Training und Validierung
### Legende:
- **TP** - True Positive
- **FP** - False Positive
- **TN** - True Negative
- **FN** - False Negative

### Metriken:
- **Categorical Accuracy** = $\frac{TP+FP}{TP+FP+TN+FN}$
- **Precision** = $\frac{TP}{TP+FP}$
- **Recall** = $\frac{TP}{TP+FN}$
- **F1 Score** = $2*\frac{Precision*Recall}{Precision+Recall}$

In [ ]:
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.FalseNegatives(),
    tf.keras.metrics.FalsePositives(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
    tfa.metrics.F1Score(num_classes=10)
]

## Training
- **Epochs**: Anzahl der Zyklen des Trainings. Gemäß des Papers wurde AlexNet in 90 Epochen trainiert
- **SGD** (Stochastic Gradient Descent): Gemäß des Papers wurde AlexNet mit SGD trainiert
- Validierungs- und Trainingdaten werden noch ermittelt (tbd - to be determined)

In [ ]:
EPOCHS = 90

# Compilieren und Trainieren
alexnet.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.01, momentum=0.9, nesterov=False, name='SGD'
    ),
    metrics=metrics,
)

history = alexnet.fit(
    epochs=EPOCHS,
          # validation and training data tbd
          validation_freq=1,
          callbacks=callbacks
)